# 1. Loading and Cleaning

## 1.1 Import libraries and python files

In [1]:
#import useful libraries 
import json
import sys
import requests
import calendar
import scipy

import pandas as pd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import seaborn as sbn
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from matplotlib.colors import LogNorm, Normalize
from sklearn.preprocessing import normalize
from datetime import datetime
%matplotlib inline 

# Import functions helper and loading functions
from data_loader import *
from helper_functions import *
%load_ext autoreload
%autoreload 2


##link to the datasets
## https://www.cs.cmu.edu/~ark/personas/
##https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download&select=movies_metadata.csv
##https://www.kaggle.com/datasets/ashirwadsangwan/imdb-dataset

## 1.2 Load the different datasets

In [2]:
##load datasets from CMU dataset
df_character = load_character().copy()
df_movie = load_movie().copy()
df_name_cluster = load_name_cluster().copy()
df_summary = load_plot_summary().copy()
df_tropes_cluster = load_tropes_cluster().copy()

In [3]:
# Replace 'id-translation.wikidata.json' with the path to your JSON file
file_path = 'id-translation.wikidata.json'

# Load the JSON file into a DataFrame
df_id_translation = pd.read_json(file_path, orient='records', lines=True)

In [4]:
# Small cleanup
df_id_translation = df_id_translation.dropna(subset=['Freebase ID'])
df_id_translation.drop_duplicates(subset=['Freebase ID'], keep='first', inplace=True)

In [5]:
# df_movie will be the reference dataframe. After being completed with additional datasets, it must never be modified
df_movie = pd.merge(df_movie, df_id_translation, on='Freebase ID', how='left')

In [6]:
#compute percentage of missing values for df_movie
values = ['Year', 'Month', 'Box office', 'Runtime', 'tconst']

def compute_missing_values(df, values):
    for variable in values :
        percentage_missing_values = (df[variable].isna().sum()/len(df[variable]))*100
        print(f"The percentage of missing values in {variable} is {format(percentage_missing_values, '.3f')}%.")

    
compute_missing_values(df_movie, values)

The percentage of missing values in Year is 8.444%.
The percentage of missing values in Month is 51.832%.
The percentage of missing values in Box office is 89.722%.
The percentage of missing values in Runtime is 25.018%.
The percentage of missing values in tconst is 9.769%.


As we can see, a lot of 'Box office' data is missing. We should add some other database to try to reduce the missing data.

## 1.3 Load addtionnal datasets and merge what we need 

In [7]:
#load imdb dataset (from kaggle)
df_imdb_movie = load_movie_imdb_kaggle()
df_imdb_rating = load_rating_imdb_kaggle() 

#merge movies with rating 
df_movie_rating = pd.merge(df_imdb_movie, df_imdb_rating, on='tconst', how='inner')

In [8]:
#merge CMU dataset with IMDB dataset
df_movie = pd.merge(df_movie, df_movie_rating[['tconst', 'averageRating', 'numVotes']], on=['tconst'], how='left')
#display(df_movie)

In [9]:
# missing Rating values
values = ['averageRating', 'numVotes']

compute_missing_values(df_movie, values)
print(f"The number of movies with missing values for 'averageRating' and 'numVotes' is {df_movie['averageRating'].isna().sum()}.")

The percentage of missing values in averageRating is 31.271%.
The percentage of missing values in numVotes is 31.271%.
The number of movies with missing values for 'averageRating' and 'numVotes' is 25561.


In [10]:
## load kaggle movie metadata
df_kaggle_movie = load_movie_kaggle()

In [11]:
#merge the datasets on the 'Name' column
#df_movie = df_movie.merge(df_kaggle_movie[['Name', 'Year', 'Month', 'Day']], on=['Name', 'Year'], how='left', suffixes=('', '_df2'))
df_movie = df_movie.merge(df_kaggle_movie[['Name', 'Year', 'tconst', 'Month', 'Day', 'budget']], on=['Name', 'Year','tconst'], how='left', suffixes=('', '_df2'))
df_movie.loc[df_movie['budget'] == '0', 'budget'] = np.nan
display(df_movie)

df_movie['budget'] = pd.to_numeric(df_movie['budget'], errors='coerce')


#print(len(df_movie[(df_movie['budget'] >0) & 
#                          (df_movie['Box office'].astype(float)>0)]  ))

,Wikipedia ID,Freebase ID,Name,Release date,Box office,Runtime,Languages (Freebase ID:name tuples),Countries (Freebase ID:name tuples),genres (Freebase ID:name tuples),Year,Month,Day,tconst,averageRating,numVotes,Month_df2,Day_df2,budget
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001,8,24,tt0228333,4.9,56854.0,8,24,28000000
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000,2,16,tt0245916,NaN,NaN,<NA>,<NA>,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988,<NA>,<NA>,tt0094806,5.6,40.0,<NA>,<NA>,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987,<NA>,<NA>,tt0094320,6.1,2888.0,<NA>,<NA>,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983,<NA>,<NA>,tt0083949,6.0,621.0,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81752,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011,3,19,tt1816585,NaN,NaN,<NA>,<NA>,NaN
81753,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011,1,21,tt1606259,6.8,3191.0,12,9,NaN
81754,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972,9,22,tt0362411,5.8,110.0,<NA>,<NA>,NaN
81755,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992,5,21,tt0113726,NaN,NaN,<NA>,<NA>,NaN


In [12]:
#use 'combine_first' to fill in the 'Box office' values from df_kaggle_movie where they are NaN in df_movie
df_movie['Month'] = df_movie['Month'].combine_first(df_movie['Month_df2'])
df_movie['Day'] = df_movie['Day'].combine_first(df_movie['Day_df2'])

#drop the extra 'Box office' column from df_kaggle_movie
df_movie.drop('Month_df2', axis=1, inplace=True)
df_movie.drop('Day_df2', axis=1, inplace=True)

In [13]:
values = ['Year', 'Month', 'Box office', 'Runtime', 'tconst', 'budget']
compute_missing_values(df_movie, values)

The percentage of missing values in Year is 8.442%.
The percentage of missing values in Month is 42.891%.
The percentage of missing values in Box office is 89.721%.
The percentage of missing values in Runtime is 25.014%.
The percentage of missing values in tconst is 9.767%.
The percentage of missing values in budget is 93.103%.


In [14]:
#load oscar reward dataset
df_oscar = load_oscar_winner()

In [15]:
#filter the dataframe to only include rows where 'winner' is True
df_winner_movie = df_oscar[df_oscar['winner'] == True]

#group by the movie name and count the winners time
df_winner_movie = df_winner_movie.groupby('Name').agg(
    num_oscars_won=pd.NamedAgg(column='winner', aggfunc='size'),
    years_won=pd.NamedAgg(column='year_ceremony', aggfunc=lambda x: list(x))
).reset_index()

#sort the movies by oscars won
df_oscar_wins = df_winner_movie.sort_values(by='num_oscars_won', ascending=False)

In [16]:
#merge CUM and oscars awards dataset
df_movie = pd.merge(df_movie, df_oscar_wins, left_on='Name', right_on='Name', how='left')

#replace NaN values in 'num_oscars_won' with 0 for movies that didn't win any oscars
df_movie['num_oscars_won'].fillna(0, inplace=True)

In [17]:
percentage_oascar_movies = ((df_movie['num_oscars_won']!=0).sum()/len(df_movie['num_oscars_won']))*100
print(f"The percentage of movies having won an oscar is {format(percentage_oascar_movies, '.3f')}%.")

The percentage of movies having won an oscar is 1.896%.


## 1.4 Cleanup of the data

In [18]:
# Drop the row with 'Year' smaller than 1800 as we have seen somme erronous movie before and there is too few movies after 2014.
df_movie = df_movie[(df_movie['Year'] >= 1800) & (df_movie['Year'] <= 2013)]

# 2. First day of the month

Let's try to prove that releasing on the 1st day of the month is a mistake

In [19]:
df_time_stamps = get_time_stamps_df(df_movie)

df_time_stamps.rename(columns={'Countries (Freebase ID:name tuples)' : 'Countries',
                                'Box office' : 'Box_office',
                              'genres (Freebase ID:name tuples)' : 'Genres',
                               'Languages (Freebase ID:name tuples)' : 'Languages'
                              }, errors="raise", inplace=True)


df_time_stamps['Genres']= df_time_stamps['Genres'].apply(lambda x : list(json.loads(x).values()))
df_time_stamps['Countries']= df_time_stamps['Countries'].apply(lambda x : list(json.loads(x).values()))
df_time_stamps['Languages']= df_time_stamps['Languages'].apply(lambda x : list(json.loads(x).values()))



df_time_stamps.drop(labels=['Freebase ID', 'Release date', 'tconst'], axis=1, inplace=True)
#display(df_time_stamps)

In [20]:
df_genre_list = df_time_stamps.copy(deep=True)
genres = get_values_column_of_list(df_genre_list, 'Genres')
sorted_genres = sorted(genres.items(), key=lambda x:x[1], reverse=True)
top_genres = [genre[0] for genre in sorted_genres[:5]]

df_country_list = df_time_stamps.copy(deep=True)
countries = get_values_column_of_list(df_country_list, 'Countries')
sorted_countries = sorted(countries.items(), key=lambda x:x[1], reverse=True)
top_countries = [country[0] for country in sorted_countries[:5]]

df_language_list = df_time_stamps.copy(deep=True)
languages = get_values_column_of_list(df_language_list, 'Languages')
sorted_languages = sorted(languages.items(), key=lambda x:x[1], reverse=True)
top_languages = [language[0] for language in sorted_languages[:3]]


def add_dummies(df, var, top_dummies):
    df_dummies = df.copy(deep=True)

    for value in top_dummies:
        df_dummies[str(value).replace(' ', '_').replace('-','_') + '_onehot'] = df_dummies[var].apply(lambda x: 1 if value in x else 0)
    df_dummies.drop(labels=var, axis=1, inplace=True)
    return df_dummies

df_time_stamps = add_dummies(df_time_stamps, 'Genres', top_genres)
df_time_stamps = add_dummies(df_time_stamps, 'Countries', top_countries)
df_time_stamps = add_dummies(df_time_stamps, 'Languages', top_languages)

In [21]:
display(df_time_stamps)

,Wikipedia ID,Name,Box_office,Runtime,Year,Month,Day,averageRating,numVotes,budget,...,Black_and_white_onehot,Thriller_onehot,United_States_of_America_onehot,United_Kingdom_onehot,India_onehot,France_onehot,Japan_onehot,English_Language_onehot,French_Language_onehot,Silent_film_onehot
0,975900,Ghosts of Mars,14010832.0,98.0,2001,8,24,4.9,56854.0,28000000.0,...,0,1,1,0,0,0,0,1,0,0
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,2000,2,16,NaN,NaN,NaN,...,0,0,1,0,0,0,0,1,0,0
5,13696889,The Gangsters,NaN,35.0,1913,5,29,NaN,NaN,NaN,...,1,0,1,0,0,0,0,1,0,1
7,10408933,Alexander's Ragtime Band,3600000.0,106.0,1938,8,16,6.8,2265.0,2000000.0,...,1,0,1,0,0,0,0,1,0,0
12,6631279,Little city,NaN,93.0,1997,4,4,5.8,1129.0,NaN,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81751,32468537,Shadow Boxing 2,NaN,132.0,2007,10,18,5.8,1066.0,NaN,...,0,0,0,0,0,0,0,1,0,0
81752,35228177,Mermaids: The Body Found,NaN,120.0,2011,3,19,NaN,NaN,NaN,...,0,0,1,0,0,0,0,1,0,0
81753,34980460,Knuckle,NaN,96.0,2011,1,21,6.8,3191.0,NaN,...,0,0,0,1,0,0,0,1,0,0
81754,9971909,Another Nice Mess,NaN,66.0,1972,9,22,5.8,110.0,NaN,...,0,0,1,0,0,0,0,1,0,0


In [38]:
import json
import statsmodels.formula.api as smf
import networkx as nx
import time
import numpy as np
import pandas as pd

def get_columns(Name, df, col):
        out = df[df['Wikipedia ID']==Name][col].values[0]
        return out

def normalize(df, var):
    return (df[var] - df[var].mean())/df[var].std()


def analyse(df, dependent_var, matching_vars, independent_var, row_id_index, onehot_vars):
    df_matching = df.copy(deep=True).dropna(subset=matching_vars + [dependent_var, independent_var] + onehot_vars)
  
    for v in matching_vars:
        df_matching[v] = df_matching[v].astype(float)
        df_matching[v] = normalize(df_matching, v)
        
        

    
    mod = smf.logit(formula=f'{dependent_var} ~  {" + ".join(matching_vars + onehot_vars)}', data=df_matching)
    res = mod.fit(maxiter=100)
    df_matching['Propensity_score'] = res.predict()

    #display(df_matching)

    # We start by creating the two groups
    treatment_group = df_matching[df_matching[dependent_var]==1]
    control_group = df_matching[df_matching[dependent_var]==0]

    # We print the number of element in each group to check that their sum is 1538 and make sure that our matching has
    # the same size as the smallest of the two groups (sanity check)
    print(len(treatment_group), len(control_group))


    G = nx.Graph()

    # Add nodes for each paper in the treatment and control groups
    G.add_nodes_from(treatment_group[row_id_index], bipartite=0)
    G.add_nodes_from(control_group[row_id_index], bipartite=1)

    # Calculate dissimilarity scores using vectorization
    treatment_scores = np.array(treatment_group['Propensity_score'])
    control_scores = np.array(control_group['Propensity_score'])

    dissimilarity_scores = np.abs(treatment_scores[:, None] - control_scores)

    start = time.time()

    for i, t_node in enumerate(treatment_group[row_id_index]):
        for j, c_node in enumerate(control_group[row_id_index]):
            G.add_edge(t_node, c_node, weight=dissimilarity_scores[i, j])

        
    end = time.time()

    print(f'Time : {end-start}')
    # This function is to help us print the final dataframe (sanity check)
    
    col = 'Propensity_score'

    # Perform minimum weight full matching
    matched_pairs = nx.bipartite.minimum_weight_full_matching(G, weight='weight', top_nodes=treatment_group[row_id_index])
    #matched_pairs = nx.max_weight_matching(G)

    # The matching function from Networkx gives a symmetric dictionary (2 times too long), so we filter it here
    filtered_edges = dict([(u, v) for u, v in matched_pairs.items() if G.nodes[u]['bipartite'] == 0 and G.nodes[v]['bipartite'] == 1])

    # Create the sanity check dataframe (shows score and track)
    df_check_matched = pd.DataFrame([(u, v, 
                                get_columns(u, treatment_group, col),
                                get_columns(v, control_group, col),
                            ) 
                            for u, v in filtered_edges.items()], 
                            columns=['Name_t', 'Name_c', col + ' u', col + ' v'])
    
    df_check_matched['matching_value'] = abs(df_check_matched['Propensity_score u']-df_check_matched['Propensity_score v'])
    
    df_check_matched = df_check_matched[df_check_matched['matching_value'] < 0.0001]

    # Create the simple dataframe with matched papers
    df_matched = df_check_matched[['Name_t', 'Name_c']]
    
    

    #display(df_check_matched)

    #print(len(df_check_matched[df_check_matched['track u'] != df_check_matched['track v']]))
    df_matched_treatment = df_matching[(df_matching[dependent_var] == 1) & 
                                    (df_matching[row_id_index].isin(df_matched['Name_t']))]

    df_matched_control = df_matching[(df_matching[dependent_var] == 0) & 
                                    (df_matching[row_id_index].isin(df_matched['Name_c']))]


    #display(df_matched_treatment)

    paired_ttest = smf.ols(formula=f'{independent_var} ~ {dependent_var}', data=pd.concat([df_matched_treatment, 
                                                                                    df_matched_control])).fit()
    print(f'Test for {independent_var} ~ {dependent_var}')
    print('')
    print(paired_ttest.summary())
    print('')
    
    return df_matched_treatment, df_matched_control


We will use propensity score for the variables : 
- Genres 
- Year
- Month
- num_oscars_won
- Countries 
- Runtime
- averageRating
- Budget
- Month Day
- Week Day


In [39]:
onehot_vars = []
onehot_vars = onehot_vars + [genre.replace(' ', '_').replace('-','_') + '_onehot' for genre in top_genres] 
onehot_vars = onehot_vars + [country.replace(' ', '_').replace('-','_') + '_onehot' for country in top_countries]
#onehot_vars = onehot_vars + [lang.replace(' ', '_').replace('-','_') + '_onehot' for lang in top_languages]


In [51]:
df_time_stamps['is_month'] = (df_time_stamps['Month'] == 7).astype(int)

matching_vars = ['Year', 'Runtime', 'budget', 'Day']


df_matched_treatement, df_matched_control = analyse(df_time_stamps, 'is_month', matching_vars, 
                                                    'Box_office', 'Wikipedia ID', onehot_vars)



Optimization terminated successfully.
         Current function value: 0.262239
         Iterations 8
288 3393
Time : 0.7141640186309814
Test for Box_office ~ is_month

                            OLS Regression Results                            
Dep. Variable:             Box_office   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     6.432
Date:                Thu, 30 Nov 2023   Prob (F-statistic):             0.0115
Time:                        14:15:06   Log-Likelihood:                -10010.
No. Observations:                 496   AIC:                         2.002e+04
Df Residuals:                     494   BIC:                         2.003e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err         

In [52]:
df_time_stamps['is_first_day'] = (df_time_stamps['Day'] == 1).astype(int)

matching_vars = ['Year', 'Runtime', 'budget', 'Month'] 


df_matched_treatement, df_matched_control = analyse(df_time_stamps, 'is_first_day', matching_vars, 
                                                    'Box_office', 'Wikipedia ID', onehot_vars)


Optimization terminated successfully.
         Current function value: 0.160637
         Iterations 60
143 3538
Time : 0.4260852336883545
Test for Box_office ~ is_first_day

                            OLS Regression Results                            
Dep. Variable:             Box_office   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.018
Date:                Thu, 30 Nov 2023   Prob (F-statistic):              0.314
Time:                        14:15:18   Log-Likelihood:                -5503.0
No. Observations:                 270   AIC:                         1.101e+04
Df Residuals:                     268   BIC:                         1.102e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err  

In [54]:
df_time_stamps['is_friday'] = (df_time_stamps['Weekday Name'] == 'Friday').astype(int)

matching_vars = ['Year', 'Runtime', 'budget', 'Day'] 

df_matched_treatement, df_matched_control = analyse(df_time_stamps, 'is_friday', matching_vars, 
                                                    'Box_office', 'Wikipedia ID', onehot_vars)



Optimization terminated successfully.
         Current function value: 0.668118
         Iterations 6
1764 1917
Time : 2.5435779094696045
Test for Box_office ~ is_friday

                            OLS Regression Results                            
Dep. Variable:             Box_office   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     12.30
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           0.000478
Time:                        14:16:07   Log-Likelihood:                -15987.
No. Observations:                 802   AIC:                         3.198e+04
Df Residuals:                     800   BIC:                         3.199e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err       

In [66]:
df_time_stamps['won_oscar'] = (df_time_stamps['num_oscars_won'] > 0).astype(int)
df_time_stamps['is_month'] = (df_time_stamps['Month'] <= 4).astype(int)


matching_vars = ['Year', 'Runtime', 'budget', 'Day', 'Weekday'] 



df_matched_treatement, df_matched_control = analyse(df_time_stamps, 'is_month', matching_vars, 
                                                    'won_oscar', 'Wikipedia ID', onehot_vars)



Optimization terminated successfully.
         Current function value: 0.588746
         Iterations 6
1602 3895
Time : 4.807483196258545
Test for won_oscar ~ is_month

                            OLS Regression Results                            
Dep. Variable:              won_oscar   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     12.81
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           0.000352
Time:                        14:21:27   Log-Likelihood:                -138.08
No. Observations:                2406   AIC:                             280.2
Df Residuals:                    2404   BIC:                             291.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          

In [72]:
df_time_stamps['is_month'] = (df_time_stamps['Month'] <= 4).astype(int)


matching_vars = ['Year', 'Runtime'] 


df_matched_treatement, df_matched_control = analyse(df_time_stamps[df_time_stamps['Year']>2005], 'is_month', matching_vars, 
                                                    'budget', 'Wikipedia ID', onehot_vars)



Optimization terminated successfully.
         Current function value: 0.622821
         Iterations 6
579 1155
Time : 0.5276849269866943
Test for budget ~ is_month

                            OLS Regression Results                            
Dep. Variable:                 budget   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     6.280
Date:                Thu, 30 Nov 2023   Prob (F-statistic):             0.0125
Time:                        14:25:18   Log-Likelihood:                -9543.6
No. Observations:                 504   AIC:                         1.909e+04
Df Residuals:                     502   BIC:                         1.910e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t  